In [5]:
import pandas as pd
cols = ['Material', 'Movement type', 'Qty in unit of entry', 'Unit of Entry', 'Order']
df = pd.concat(pd.read_excel(
    '../data/TONG.XLSX', sheet_name=None, usecols=cols ,converters={
            'Material': str,
            'Order': str
        }
    ), ignore_index=True
)

df = df.loc[
    df['Movement type'].isin([101, 102, 261, 262]) &
    ~pd.isna(df['Order'])
]
sx = df.loc[df['Movement type'].isin([101, 102])]
nvl = df.loc[df['Movement type'].isin([261, 262])]
sx = sx.groupby(
    ['Material', 'Unit of Entry', 'Order']).agg(
        {'Qty in unit of entry': 'sum'}
)
sx.reset_index(inplace=True)
sx.rename(columns={
    'Material': 'Product',
    'Unit of Entry': 'Product Unit',
    'Qty in unit of entry': 'Product Quantity'
}, inplace=True)
sx = sx.loc[sx['Product Quantity'] != 0]

nvl = nvl.groupby(
    ['Material', 'Unit of Entry', 'Order']).agg(
        {'Qty in unit of entry': 'sum'}
)
nvl.reset_index(inplace=True)
nvl.rename(columns={
    'Unit of Entry': 'Material Unit',
    'Qty in unit of entry': 'Material Quantity'
}, inplace=True)

nvl['Material Quantity'] = -nvl['Material Quantity']
sx.drop_duplicates(inplace=True)
result = pd.merge(sx, nvl, how="left", on='Order', sort=False,
                          indicator=False, validate=None)
result = result.groupby(['Product', 'Product Unit', 'Material', 'Material Unit']).agg(
            {'Product Quantity': 'sum', 'Material Quantity': 'sum'})
result.reset_index(inplace=True)
result['BOM'] = result['Material Quantity'] / result['Product Quantity']


In [1]:
import pandas as pd
df = pd.concat(pd.read_excel(
    '../data/2022-03-24_BOM_KG3.xlsx', sheet_name=None,
    ), ignore_index=True
)
df = df[df['Usage'].notna()]
df.to_excel('../output/KG3.xlsx')


df = pd.read_excel('../data/BOM winnercom.xlsx', header=6, usecols=['Mã mẹ', 'Mã con', 'Actual BOM'], converters={'Actual BOM': float})
df

result = df.groupby(['Mã mẹ', 'Mã con'], sort=False).agg(
            {'Actual BOM': 'sum'})
result.to_excel('../output/winnercom.xlsx')


import pandas as pd

df = pd.read_excel('../data/BOM 2021.xlsx', header=11)
cols = df.columns[18:]
df = df[cols]
df.set_index('Material Name', inplace=True)
trans = df.transpose()
cols = trans.columns
trans.reset_index(inplace=True)
trans = trans.iloc[2:]
cols = [x for x in trans.columns if not pd.isna(x)]
trans = trans[cols]

trans.melt(id_vars=["index"], ignore_index = False).to_excel('../output/hope.xlsx')
result= trans.set_index(["index"]).stack()
result = result.reset_index()
df.reset_index(inplace=True)
df = df[['Material Name', 'Unit.1']]
end = pd.merge(
    result,
    df,
    how="left",
    on='Material Name',
    sort=False,
    indicator=False,
    validate=None,
)
end = end.drop(end[end[0] == 0].index)


d:\Projects\Excel-to-SQL-snippets\env\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Projects\Excel-to-SQL-snippets\env\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
